In [1]:
from imutils import paths
import os
import librosa
import numpy as np

def getMFCC(i):
  signal, sr = librosa.load(i, sr=16000)

  # MFCC를 통한 특징 벡터 추출 (n_mfcc=36)
  MFCCs = librosa.feature.mfcc(signal, sr, n_fft=400, hop_length=160, n_mfcc=36)
  return MFCCs

forder = '/content/drive/MyDrive/cut4/cut4/'
files = paths.list_files(forder)

dataset=[]
label=[] 

for i in files:
  mfcc=getMFCC(i)
  dataset.append(mfcc.reshape(-1)) # 1차원 배열로 반환
  label.append((os.path.basename(i)).split('_')[1]) # 파일의 이름에서 감정을 나타내는 부분을 추출해서 저장


# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

encoder.fit(label)
y = encoder.transform(label)


from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 학습 데이터와 훈련 데이터 split
X_train, X_test, y_train, y_test = train_test_split(dataset, y, test_size = 0.2, random_state = 42, shuffle = True)

# 커널 설정
clf = SVC(C=10, kernel = 'rbf', probability=True)

# MinMax Scaler를 통해 특징 벡터 전처리
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 조정된 데이터로 SVM 학습
clf.fit(X_train_scaled, y_train)

# 스케일 조정된 테스트 세트의 정확도
print(clf.score(X_test_scaled, y_test))

#import pickle
from sklearn.externals import joblib

# 학습한 모델 파일로 저장
joblib.dump(clf, 'model.pkl')


# 파일로 저장된 모델 불러와서 예측
clf_from_joblib = joblib.load('model.pkl') 
clf_from_joblib.predict(X_test_scaled)



0.8189189189189189


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


array([4, 0, 0, 1, 4, 3, 0, 1, 2, 1, 0, 4, 0, 4, 2, 4, 0, 2, 4, 4, 4, 1,
       3, 1, 4, 2, 0, 2, 2, 2, 1, 0, 2, 2, 4, 2, 3, 2, 3, 0, 1, 2, 2, 1,
       2, 3, 2, 2, 2, 4, 2, 2, 0, 4, 2, 0, 4, 4, 4, 4, 0, 3, 4, 0, 4, 3,
       4, 2, 0, 0, 2, 1, 2, 2, 4, 0, 1, 4, 0, 2, 1, 0, 4, 4, 2, 2, 1, 4,
       2, 2, 4, 0, 4, 3, 3, 4, 0, 1, 0, 4, 4, 2, 3, 3, 4, 0, 4, 2, 1, 2,
       4, 0, 2, 1, 4, 4, 1, 2, 3, 2, 0, 4, 0, 4, 4, 3, 3, 1, 4, 1, 0, 0,
       4, 0, 0, 1, 0, 1, 3, 2, 0, 2, 4, 1, 4, 4, 1, 1, 2, 4, 2, 4, 4, 2,
       0, 3, 3, 2, 2, 1, 0, 4, 0, 3, 0, 4, 4, 3, 3, 2, 2, 4, 0, 2, 4, 0,
       0, 3, 3, 2, 4, 4, 0, 3, 2, 3, 4, 1, 4, 1, 3, 1, 1, 4, 1, 4, 3, 3,
       0, 4, 1, 2, 4, 4, 0, 1, 0, 4, 2, 3, 3, 1, 0, 1, 4, 0, 3, 1, 3, 3,
       0, 2, 2, 2, 0, 4, 4, 3, 4, 0, 4, 1, 3, 0, 4, 4, 2, 2, 0, 3, 0, 4,
       0, 0, 2, 2, 3, 4, 4, 1, 1, 4, 4, 4, 1, 0, 4, 4, 0, 0, 4, 2, 4, 3,
       3, 0, 3, 4, 0, 3, 1, 2, 4, 2, 0, 1, 4, 3, 1, 2, 3, 1, 0, 4, 4, 4,
       2, 2, 4, 2, 1, 4, 2, 4, 0, 3, 2, 2, 2, 2, 2,